In [1]:
import csv
import collections

**My notes from the book Spark - The Definitive Guide**

# Fundamentals

SparkSession is the new entry point to interact with Spark functionalities. Befor Spark 2.0.0, we needed to create different SparkContext for these functionalities. When starting Spark on an interactive mode, a SparkSession is automatically created. In this notebook, we need to do that manually.

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
# local means that driver and executers run on your individual
# computer instead of a cluster. 
spark = SparkSession.builder.master("local").appName("practice") \
        .getOrCreate()

In [2]:
print(spark)

In [4]:
spark

**DataFrame**  
Table of data with rows and columns. Similar to a DataFrame, but distributed into partitions (groups of rows on different clusters). Creating a df from a range of numbers is easy:

In [3]:
my_range = spark.range(1000).toDF("number")

In [9]:
# narrow transformation (fast)
divis_by2 = my_range.where("number % 2 = 0")
# wide transformation + collect (slow since it uses shuffling)
divis_by2.count()

500

Reading from CSV file (reading data in Spark is also a transformation):

In [10]:
# using schema inference 
r_packages = spark.read.option("inferSchema", "true").option("header", "true") \
                  .csv("./data/r_packages_log.csv")

Visualizing an execution plan:

In [15]:
r_packages.sort("size").explain()

== Physical Plan ==
*(2) Sort [size#30 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(size#30 ASC NULLS FIRST, 200)
   +- *(1) FileScan csv [date#28,time#29,size#30,r_version#31,r_arch#32,r_os#33,package#34,version#35,country#36,ip_id#37] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/USUARIO/Dropbox/ds_code/git-projects/spark-toolbox/data/r_packag..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<date:timestamp,time:string,size:int,r_version:string,r_arch:string,r_os:string,package:str...


In [16]:
# 200 is the standard number of partitions
spark.conf.set("spark.sql.shuffle.partitions", "200")
r_packages.sort("size").take(2)

[Row(date=datetime.datetime(2015, 12, 12, 0, 0), time='19:06:56', size=504, r_version='NA', r_arch='NA', r_os='NA', package='httpRequest', version='0.0.5', country='CN', ip_id=1133),
 Row(date=datetime.datetime(2015, 12, 12, 0, 0), time='20:33:02', size=504, r_version='NA', r_arch='NA', r_os='NA', package='granova', version='1.4', country='CN', ip_id=5331)]

## Spark SQL
SQL queries and DataFrame code both compile to Spark code, so there is no difference in performance. Before using SQL commnads, we need to create a table or a view. We can do this from a dataframe:

In [17]:
r_packages.createOrReplaceTempView("r_packages")

Now we can execute SQL statements:

In [18]:
query = spark.sql("""
SELECT size, count(1) 
FROM r_packages
GROUP BY size
""").explain()

== Physical Plan ==
*(2) HashAggregate(keys=[size#30], functions=[count(1)])
+- Exchange hashpartitioning(size#30, 200)
   +- *(1) HashAggregate(keys=[size#30], functions=[partial_count(1)])
      +- *(1) FileScan csv [size#30] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/USUARIO/Dropbox/ds_code/git-projects/spark-toolbox/data/r_packag..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<size:int>


We can import several transformation functions

In [21]:
from pyspark.sql.functions import max

r_packages.select(max("size")).take(1)

[Row(max(size)=68736865)]

More complex query using SQL (top 5 average package sizes):

In [22]:
spark.sql("""
SELECT country, AVG(size) as avg_size
FROM r_packages
GROUP BY country
ORDER BY avg_size DESC
LIMIT 5
""").show()

+-------+------------------+
|country|          avg_size|
+-------+------------------+
|     IQ|         4306891.6|
|     IS| 3690250.714285714|
|     SV|         3557710.1|
|     KW|2896512.4444444445|
|     GE|        2672563.25|
+-------+------------------+



Or using Spark functional paradigm:

In [23]:
from pyspark.sql.functions import desc

r_packages.groupBy("country").avg("size").withColumnRenamed("avg(size)", "avg_size") \
          .sort(desc("avg_size")).limit(5).show()

+-------+------------------+
|country|          avg_size|
+-------+------------------+
|     IQ|         4306891.6|
|     IS| 3690250.714285714|
|     SV|         3557710.1|
|     KW|2896512.4444444445|
|     GE|        2672563.25|
+-------+------------------+



In [24]:
r_packages.groupBy("country").avg("size").withColumnRenamed("avg(size)", "avg_size") \
          .sort(desc("avg_size")).limit(5).explain()

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[avg_size#187 DESC NULLS LAST], output=[country#36,avg_size#187])
+- *(2) HashAggregate(keys=[country#36], functions=[avg(cast(size#30 as bigint))])
   +- Exchange hashpartitioning(country#36, 200)
      +- *(1) HashAggregate(keys=[country#36], functions=[partial_avg(cast(size#30 as bigint))])
         +- *(1) FileScan csv [size#30,country#36] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/USUARIO/Dropbox/ds_code/git-projects/spark-toolbox/data/r_packag..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<size:int,country:string>


## Streaming

In [30]:
import pathlib

The 'by-day' folder contains one csv file for every day in the dataset:

In [31]:
DATA_PATH = "../../Spark-The-Definitive-Guide/data/retail-data/by-day/*.csv"

In [32]:
# Infering schema from several file
static_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true") \
                 .load(DATA_PATH)
# Create view/table
static_df.createOrReplaceTempView("retail_data")
static_schema = static_df.schema

Example of query with window function (windows over time):

In [88]:
from pyspark.sql.functions import window, column, desc, col
spark.conf.set("spark.sql.shuffle.partitions", "5")
static_df.selectExpr("CustomerId", "(UnitPrice * Quantity) as total_cost", "InvoiceDate") \
         .groupBy("CustomerId", window(col("InvoiceDate"), "1 day")).sum("total_cost").show(5)

+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|   14075.0|[2011-12-04 19:00...|316.78000000000003|
|   18180.0|[2011-12-04 19:00...|            310.73|
|   15358.0|[2011-12-04 19:00...| 830.0600000000003|
|   15392.0|[2011-12-04 19:00...|304.40999999999997|
|   15290.0|[2011-12-04 19:00...|263.02000000000004|
+----------+--------------------+------------------+
only showing top 5 rows



Now using readStream:

In [91]:
streaming_df = spark.readStream.schema(static_schema) \
    .option("maxFilesPerTrigger", 1) \
    .format("csv") \
    .option("header", "true") \
    .load(DATA_PATH)

In [93]:
streaming_df.isStreaming

True

Setting up the query:

In [95]:
purchase_by_costumer_per_hour = streaming_df\
    .selectExpr(
      "CustomerId",
      "(UnitPrice * Quantity) as total_cost",
      "InvoiceDate")\
    .groupby(
      col("CustomerId"), window("InvoiceDate", "1 day"))\
    .sum("total_cost")

We process the stream data and write it on a in memory table. The table is update after each trigger (defined above).

In [96]:
purchase_by_costumer_per_hour.writeStream\
    .format("memory")\
    .queryName("customer_purchases")\
    .outputMode("complete")\
    .start()

Testing the stream:

In [122]:
spark.sql("""
SELECT *
FROM customer_purchases
ORDER BY 'sum(total_cost)' DESC
""").show(5)

+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|   15290.0|[2011-02-21 19:00...|             -1.65|
|   14911.0|[2011-03-10 19:00...|               0.0|
|   15208.0|[2010-12-20 19:00...|              65.4|
|   15694.0|[2011-03-15 20:00...|            584.76|
|   12921.0|[2011-03-29 20:00...|-87.30000000000001|
+----------+--------------------+------------------+
only showing top 5 rows



## Machine Learning

Loading Data (same data used on the streaming example)

In [126]:
DATA_PATH = "../../Spark-The-Definitive-Guide/data/retail-data/by-day/*.csv"
# Infering schema from several file
static_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true") \
                 .load(DATA_PATH)

In [125]:
static_df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



Preprocessing our data (withColumn is used to add a new column):

In [131]:
from pyspark.sql.functions import date_format, col
prepared_df = static_df\
.na.fill(0)\
.withColumn("day_of_week", date_format("InvoiceDate", "EEEE"))\
.coalesce(5)

Spliting the data into training and test sets:

In [134]:
train_df = prepared_df.where("InvoiceDate < '2011-07-01'")
train_df = prepared_df.where("InvoiceDate >= '2011-07-01'")

Getting encoder for day of the week column:

In [148]:
# Simple Label Encoder (changes to integers)
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer()\
.setInputCol("day_of_week")\
.setOutputCol("day_of_week_index")

# One Hot Encoder (works on top of the previous transformation)
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder()\
.setInputCol("day_of_week_index")\
.setOutputCol("day_of_week_encoded")

We need to assemble the columns we will use into a vector (ML algorithms in Spark take as input a Vector type):

In [149]:
from pyspark.ml.feature import VectorAssembler

vector_assembler = VectorAssembler()\
.setInputCols(["UnitPrice", "Quantity", "day_of_week_encoded"])\
.setOutputCol("features")

Building the pipeline:

In [150]:
from pyspark.ml import Pipeline

transformation_pipeline = Pipeline()\
.setStages([indexer, encoder, vector_assembler])

Fitting the pipeline:

In [151]:
fitted_pipeline = transformation_pipeline.fit(train_df)

Applying to the training data:

In [152]:
transformed_training = fitted_pipeline.transform(train_df)

Training KMeans with caching:

In [157]:
transformed_training.cache()

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: timestamp, UnitPrice: double, CustomerID: double, Country: string, day_of_week: string, day_of_week_index: double, day_of_week_encoded: vector, features: vector]

In [158]:
from pyspark.ml.clustering import KMeans

In [160]:
kmeans = KMeans()\
.setK(20)\
.setSeed(1)

Naming patter in Spark: kmeans for untrained model and kmeans_model for trained version. Let's train the model:

In [161]:
kmeans_model = kmeans.fit(transformed_training)

Evaluating performance:

In [162]:
# Horrible in this case -> we need to scale the date for KNN
kmeans_model.computeCost(transformed_training)

98518684.09312333

# Structured API (4)

**Review**: The user specifies multiple *transformations* that are build on an acyclic graph of instructions. An *action* begins the process of executing the graph of instructions. Spark breaks down into stages and executes across the cluster. The logical structures for transformations and actions are DataFrames and Datasets.   
**Note**: Tables and views are basically the same thing as DataFrames. We just use SQL against them instead.

Declaring column type in python:

In [40]:
from pyspark.sql.types import *
b = ByteType()

Check pg.59 of the book to find equivalence between Python data types and Spark's.

# Basic Structured Operations (5)

DataFrames consist of records of type Row, and a number of columns (logical constructions that represent value computed on a per-record basis by means of an expression). Schemas define the name and type of data in each column. Partition is the layout of the DF across the cluster. Partitioning schema defines how that is allocated.

Creating a DataFrame:

In [5]:
DATA_PATH = "../../Spark-The-Definitive-Guide/data/flight-data/json/2015-summary.json"
df = spark.read.format("json").load(DATA_PATH)

## Schema

Visualizing the schema:

In [6]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



A schema is composed of StructType(list or tuple) of StructFields:

In [7]:
df.schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

**Tip**: define your schemas manually in production, instead of using schema-on-read. This is more robust and efficient. We have to use Spark data types for this:

In [10]:
from pyspark.sql.types import StructField, StructType, StringType, LongType

my_schema = StructType([
    StructField("DEST_COUNTRY_NAME", StringType(), True),
    StructField("ORIGIN_COUNTRY_NAME", StringType(), True),
    StructField("count", LongType(), False, metadata={"hello":"world"})
])

df = spark.read.format("json").schema(my_schema).load(DATA_PATH)

In [11]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



## DataFrame Transformations

Registering a temporary view so we can use SQL:

In [23]:
df.createOrReplaceTempView("dfTable")

**select and selectExpr**

In [28]:
df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(2)

+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
+-----------------+-------------------+
only showing top 2 rows



In [34]:
df.selectExpr("DEST_COUNTRY_NAME AS destination", "ORIGIN_COUNTRY_NAME").show(2)

+-------------+-------------------+
|  destination|ORIGIN_COUNTRY_NAME|
+-------------+-------------------+
|United States|            Romania|
|United States|            Croatia|
+-------------+-------------------+
only showing top 2 rows



We can use any valid non-aggregating SQL statement:

In [40]:
df.selectExpr("*", 
              "(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry")\
             .show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



Or aggregations over the entire DF:

In [45]:
df.selectExpr("avg(count)", "count(distinct(DEST_COUNTRY_NAME))").show()

+-----------+---------------------------------+
| avg(count)|count(DISTINCT DEST_COUNTRY_NAME)|
+-----------+---------------------------------+
|1770.765625|                              132|
+-----------+---------------------------------+



**Literals** values coming from Python

In [54]:
from pyspark.sql.functions import lit, expr

a=10
df.select(expr("*"), lit(a).alias("One")).show(2)

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15| 10|
|    United States|            Croatia|    1| 10|
+-----------------+-------------------+-----+---+
only showing top 2 rows



**Remove** column (you can also use a SELECT statement):

In [58]:
# The transformation is not inplace=True
df.drop("ORIGIN_COUNTRY_NAME").show(2)

+-----------------+-----+
|DEST_COUNTRY_NAME|count|
+-----------------+-----+
|    United States|   15|
|    United States|    1|
+-----------------+-----+
only showing top 2 rows



**cast** (changing column type):

In [62]:
# withColumn adds or replaces and existing column
df.withColumn("count2", col("count").cast("string")).show(2)

+-----------------+-------------------+-----+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|count2|
+-----------------+-------------------+-----+------+
|    United States|            Romania|   15|    15|
|    United States|            Croatia|    1|     1|
+-----------------+-------------------+-----+------+
only showing top 2 rows



**filter** rows

In [69]:
df.where("count<2").show(2)
#df.filter()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [75]:
df.where(col("DEST_COUNTRY_NAME")=='United States').show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [76]:
df.where("count<2").where(col("DEST_COUNTRY_NAME")=='United States').show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



**unique rows**

In [80]:
df.select("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").distinct().count()

256

In [81]:
df.select("ORIGIN_COUNTRY_NAME").distinct().count()

125

**sampling and random split**

In [84]:
df.sample(withReplacement=False, fraction=0.5, seed=5).count()

126

In [89]:
# proportions will be normalized if not adding up to 1
df.randomSplit([0.25, 0.75], seed=5)[0].count()

60

In [88]:
df.randomSplit([0.25, 0.75], seed=5)[1].count()

196

**concatenating**

In [126]:
# creating a new DF
from pyspark.sql import Row

schema = df.schema
newRows = [
    Row("New Country", "Other Country", 5),
    Row("New Country 2", "Other Country 2", 5)
]
newDF = spark.createDataFrame(newRows, schema)

In [127]:
df.union(newDF).where(col("ORIGIN_COUNTRY_NAME") == "Other Country").show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|      New Country|      Other Country|    5|
+-----------------+-------------------+-----+



**sorting**   
There are several ways to do it. asc() is used to make the command explict, but it is the standard behavior. Use asc_nulls_first, desc_nulls_last, etc. to specify where you want the null values.

In [152]:
from pyspark.sql.functions import desc, asc

df.sort("count").show(2)
df.orderBy(col("count").desc()).show(2)
df.orderBy(expr("count desc")).show(2)
df.orderBy(col("count").desc(), col("DEST_COUNTRY_NAME").asc()).show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Moldova|      United States|    1|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|    United States|             Canada|  8483|
+-----------------+-------------------+------+
only showing top 2 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Moldova|      United States|    1|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| coun

**repartitioning**

In [167]:
# get current number of partitions
df.rdd.getNumPartitions()
# repartition by number 
df.repartition(4)
# repartition by column (useful if we filter by some column frequently)
df.repartition(5, col("DEST_COUNTRY_NAME"))
# coalesce
df.coalesce(2)

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

# Different Types of Data (6)

In [9]:
DATA_PATH = "../../Spark-The-Definitive-Guide/data/retail-data/by-day/2010-12-01.csv"
df = spark.read.format("csv").option("header", "true")\
          .option("inferSchema", "true")\
          .load(DATA_PATH)
df.createOrReplaceTempView("dfTable")

In [13]:
df.count()

3108

**converting types**

In [102]:
from pyspark.sql.functions import lit

df.select(lit(5), lit("5")).show(5)

+---+---+
|  5|  5|
+---+---+
|  5|  5|
|  5|  5|
|  5|  5|
|  5|  5|
|  5|  5|
+---+---+
only showing top 5 rows



**booleans and filtering**

Example with or:

In [38]:
from pyspark.sql.functions import instr, col

priceFilter = col("UnitPrice") > 600
descripFilter = instr(df.Description, "POSTAGE") >= 1
df.where(df.StockCode.isin("DOT")).where(priceFilter | descripFilter).show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



**working with numbers**

In [51]:
from pyspark.sql.functions import expr, pow

expression = pow(col("Quantity") * col("UnitPrice"), 2) + 5
df.select(expr("CustomerId") ,expression.alias("Result")).show(2)

+----------+------------------+
|CustomerId|            Result|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



Correlation:

In [53]:
from pyspark.sql.functions import corr

df.stat.corr("Quantity", "UnitPrice")
df.select(corr("Quantity", "UnitPrice")).show()

+-------------------------+
|corr(Quantity, UnitPrice)|
+-------------------------+
|     -0.04112314436835551|
+-------------------------+



Describe like in pandas:

In [55]:
df.describe().show()

+-------+-----------------+------------------+--------------------+------------------+------------------+------------------+--------------+
|summary|        InvoiceNo|         StockCode|         Description|          Quantity|         UnitPrice|        CustomerID|       Country|
+-------+-----------------+------------------+--------------------+------------------+------------------+------------------+--------------+
|  count|             3108|              3108|                3098|              3108|              3108|              1968|          3108|
|   mean| 536516.684944841|27834.304044117645|                null| 8.627413127413128| 4.151946589446603|15661.388719512195|          null|
| stddev|72.89447869788873|17407.897548583845|                null|26.371821677029203|15.638659854603892|1854.4496996893627|          null|
|    min|           536365|             10002| 4 PURPLE FLOCK D...|               -24|               0.0|           12431.0|     Australia|
|    max|          C

**strings**

Regular Expressions: regexp_extract and regexp_replace

In [70]:
from pyspark.sql.functions import regexp_extract, regexp_replace, translate

regex_string = 'BLACK|WHITE|RED|GREEN|BLUE'
df.select(regexp_replace(col("Description"), regex_string, "AAAA").alias("Changed Sting")).show(3)

+--------------------+
|       Changed Sting|
+--------------------+
|AAAA HANGING HEAR...|
|  AAAA METAL LANTERN|
|CREAM CUPID HEART...|
+--------------------+
only showing top 3 rows



Translate is easier for character substitution:

In [75]:
df.select(col("Description"), translate(col("Description"), 'LEET', '1337').alias('Changed Column')).show(2)

+--------------------+--------------------+
|         Description|      Changed Column|
+--------------------+--------------------+
|WHITE HANGING HEA...|WHI73 HANGING H3A...|
| WHITE METAL LANTERN| WHI73 M37A1 1AN73RN|
+--------------------+--------------------+
only showing top 2 rows



Function generating columns with booleans:

In [88]:
from pyspark.sql.functions import locate

simple_colors = ["black", "red", "white"]
def color_locator(column, color_string):
    return locate(color_string.upper(), column)\
           .cast("boolean").alias("is_"+color_string)

selected_columns = [color_locator(df.Description, c) for c in simple_colors]
# trick to add other columns when using select
selected_columns.append(expr("*"))

In [93]:
df.select(*selected_columns).where(expr("is_white or is_red")).select("Description").show(3, False)

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 3 rows



**dates**

In [100]:
from pyspark.sql.functions import to_date

dateFormat = 'yyyy-dd-MM'
cleanDateDF = spark.range(1).select(
    to_date(lit('2017-12-11'), dateFormat).alias('date'),
    to_date(lit('2017-20-11'), dateFormat).alias('date2')
    
)

In [101]:
cleanDateDF.show()

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-11-20|
+----------+----------+



**null**

In [112]:
spark.sql("""
SELECT
    ifnull(null, 'return_value'),
    nullif('value', 'value'),
    nvl(null, 'return_value'),
    nvl2('not_null', 'return_value', 'else_value')
""").show()

+----------------------------+------------------------+-------------------------+----------------------------------------------+
|ifnull(NULL, 'return_value')|nullif('value', 'value')|nvl(NULL, 'return_value')|nvl2('not_null', 'return_value', 'else_value')|
+----------------------------+------------------------+-------------------------+----------------------------------------------+
|                return_value|                    null|             return_value|                                  return_value|
+----------------------------+------------------------+-------------------------+----------------------------------------------+



**structs**

In [115]:
from pyspark.sql.functions import struct

complexDF = df.select(struct("Description", "InvoiceNo").alias("complex"))
complexDF.createOrReplaceTempView("compleDF")

In [124]:
complexDF.select(col("complex").getField("Description")).show(2)
complexDF.select("complex.Description").show(2)
complexDF.select("complex.*").show(2)

+--------------------+
| complex.Description|
+--------------------+
|WHITE HANGING HEA...|
| WHITE METAL LANTERN|
+--------------------+
only showing top 2 rows

+--------------------+
|         Description|
+--------------------+
|WHITE HANGING HEA...|
| WHITE METAL LANTERN|
+--------------------+
only showing top 2 rows

+--------------------+---------+
|         Description|InvoiceNo|
+--------------------+---------+
|WHITE HANGING HEA...|   536365|
| WHITE METAL LANTERN|   536365|
+--------------------+---------+
only showing top 2 rows



**arrays**: very useful when splitting information in columns

In [135]:
from pyspark.sql.functions import split, size, array_contains

df.select(split(col("Description"), " ").alias("array_col")).selectExpr("array_col[0]").show(4)

df.select(size(split(col("Description"), " ")).alias("size")).show(4)

df.select(array_contains(split(col("Description"), " "), "WHITE").alias("WHITE?")).show(4)


+------------+
|array_col[0]|
+------------+
|       WHITE|
|       WHITE|
|       CREAM|
|     KNITTED|
+------------+
only showing top 4 rows

+----+
|size|
+----+
|   5|
|   3|
|   5|
|   6|
+----+
only showing top 4 rows

+------+
|WHITE?|
+------+
|  true|
|  true|
| false|
| false|
+------+
only showing top 4 rows



**explode**

In [153]:
from pyspark.sql.functions import split, explode

df.withColumn("splitted", split(col("Description"), " "))\
  .withColumn("exploded", explode(col("splitted")))\
  .select("Description", "InvoiceNo", "exploded").show(3)

+--------------------+---------+--------+
|         Description|InvoiceNo|exploded|
+--------------------+---------+--------+
|WHITE HANGING HEA...|   536365|   WHITE|
|WHITE HANGING HEA...|   536365| HANGING|
|WHITE HANGING HEA...|   536365|   HEART|
+--------------------+---------+--------+
only showing top 3 rows



**maps**

In [160]:
from pyspark.sql.functions import create_map

df.select(create_map(col("Description"), col("UnitPrice"))).show(2, False)

+--------------------------------------------+
|map(Description, UnitPrice)                 |
+--------------------------------------------+
|[WHITE HANGING HEART T-LIGHT HOLDER -> 2.55]|
|[WHITE METAL LANTERN -> 3.39]               |
+--------------------------------------------+
only showing top 2 rows



## JSON

In [166]:
jsonDF = spark.range(1).selectExpr("""
'{"myJSONKey" : {"myJSONValue" : [1,2,3]}}' as jsonString

""")

In [167]:
jsonDF.show()

+--------------------+
|          jsonString|
+--------------------+
|{"myJSONKey" : {"...|
+--------------------+



Single level or multiple levels access:

In [194]:
from pyspark.sql.functions import get_json_object, json_tuple

jsonDF.select(
    # multiple levels access
    get_json_object(col("jsonString"), "$.myJSONKey.myJSONValue[1]").alias("column"),
    # One level of nesting
    json_tuple(col("jsonString"), "myJSONKey")
).show()

+------+--------------------+
|column|                  c0|
+------+--------------------+
|     2|{"myJSONValue":[1...|
+------+--------------------+



Changing structs to json:

In [206]:
df.select(struct('InvoiceNo', 'Description').alias("myStruct"))\
  .selectExpr("to_json(myStruct) as JSON").show(2, False)

+-------------------------------------------------------------------------+
|JSON                                                                     |
+-------------------------------------------------------------------------+
|{"InvoiceNo":"536365","Description":"WHITE HANGING HEART T-LIGHT HOLDER"}|
|{"InvoiceNo":"536365","Description":"WHITE METAL LANTERN"}               |
+-------------------------------------------------------------------------+
only showing top 2 rows



## User-Defined Functions (UDF)

It is recommended to write custom functions using Scala.

In [214]:
udfExampleDF = spark.range(5).toDF("num")

def power3(double_value):
    return double_value ** 3

# we need to register the custom function 
from pyspark.sql.functions import udf, col
power3udf = udf(power3)
# applying the custom function
udfExampleDF.select(power3udf(col("num"))).show()


+-----------+
|power3(num)|
+-----------+
|          0|
|          1|
|          8|
|         27|
|         64|
+-----------+



# Aggregations (7)

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
# local meansthat driver and executers run on your individual
# computer instead of a cluster. 
spark = SparkSession.builder.master("local").appName("practice") \
        .getOrCreate()

Loading the data:

In [2]:
DATA_PATH = "../../Spark-The-Definitive-Guide/data/retail-data/all/*.csv"

In [3]:
df = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load(DATA_PATH)\
    .coalesce(5)

In [4]:
# for rapid access
df.cache()
df.createOrReplaceTempView("dfTable")

## Aggregation Functions

Check the chapter for more examples on: count, countDistinct, approx_count_distinct, first/last, min/max, sum, sumDistinct, avg, variance (sample and population - be careful), skewness, kurtosis, covariance/correlation

**sets and lists**

In [14]:
from pyspark.sql.functions import collect_set, collect_list

df.agg(collect_set("Country"), collect_list("Country")).show()

+--------------------+---------------------+
|collect_set(Country)|collect_list(Country)|
+--------------------+---------------------+
|[Portugal, Italy,...| [United Kingdom, ...|
+--------------------+---------------------+



## Grouping

In [19]:
df.groupBy("InvoiceNo", "CustomerId").count().show(5)

+---------+----------+-----+
|InvoiceNo|CustomerId|count|
+---------+----------+-----+
|   536846|     14573|   76|
|   537026|     12395|   12|
|   537883|     14437|    5|
|   538068|     17978|   12|
|   538279|     14952|    7|
+---------+----------+-----+
only showing top 5 rows



**using agg and expressions**

In [25]:
from pyspark.sql.functions import count

df.groupBy("InvoiceNo").agg(
    count("Quantity").alias("quant"),
    expr("count(Quantity)")
).show(5)

+---------+-----+---------------+
|InvoiceNo|quant|count(Quantity)|
+---------+-----+---------------+
|   536596|    6|              6|
|   536938|   14|             14|
|   537252|    1|              1|
|   537691|   20|             20|
|   538041|    1|              1|
+---------+-----+---------------+
only showing top 5 rows



In [26]:
df.groupBy("InvoiceNo").agg(
    expr("stddev_pop(Quantity)").alias("SD"),
    expr("count(Quantity)")
).show(5)

+---------+------------------+---------------+
|InvoiceNo|                SD|count(Quantity)|
+---------+------------------+---------------+
|   536596|1.1180339887498947|              6|
|   536938|20.698023172885524|             14|
|   537252|               0.0|              1|
|   537691| 5.597097462078001|             20|
|   538041|               0.0|              1|
+---------+------------------+---------------+
only showing top 5 rows



## Window Functions

In [8]:
df.select("InvoiceDate").show(1)

+--------------+
|   InvoiceDate|
+--------------+
|12/1/2010 8:26|
+--------------+
only showing top 1 row



In [12]:
dfWithDate.select("date").show(1)

+----------+
|      date|
+----------+
|2010-12-01|
+----------+
only showing top 1 row



In [53]:
from pyspark.sql.functions import col, to_date, desc, max, dense_rank, rank
from pyspark.sql.window import Window

dfWithDate = df.withColumn("date", to_date(col("InvoiceDate"), "MM/d/yyyy H:mm"))
# we want the rank for customer for each day
# we need this for rank
windowSpec = Window.partitionBy("CustomerId", "date")\
                   .orderBy(desc("Quantity"))\
                   .rowsBetween(Window.unboundedPreceding, Window.currentRow)

# we are creating unresolved columns ahead of time below
maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)

purchaseDenseRank = dense_rank().over(windowSpec)
purchaseRank = rank().over(windowSpec)

In [56]:
dfWithDate.where("CustomerId IS NOT NULL")\
          .orderBy("CustomerId")\
          .select(col("CustomerId"), 
                  col("date"), col("Quantity"),
                  purchaseRank.alias("quantityRank"),
                  purchaseDenseRank.alias("quantityDenseRank"),
                  maxPurchaseQuantity.alias("maxPurchaseQuantity")
                  ).show()

+----------+----------+--------+------------+-----------------+-------------------+
|CustomerId|      date|Quantity|quantityRank|quantityDenseRank|maxPurchaseQuantity|
+----------+----------+--------+------------+-----------------+-------------------+
|     12346|2011-01-18|   74215|           1|                1|              74215|
|     12346|2011-01-18|  -74215|           2|                2|              74215|
|     12347|2010-12-07|      36|           1|                1|                 36|
|     12347|2010-12-07|      30|           2|                2|                 36|
|     12347|2010-12-07|      24|           3|                3|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|             

## Grouping Sets

In [58]:
dfNoNull = dfWithDate.drop()
dfNoNull.createOrReplaceTempView("dfNoNull")

In [63]:
spark.sql("""SELECT CustomerId, stockCode, sum(Quantity) FROM dfNoNull
GROUP BY customerId, stockCode GROUPING SETS((customerId, stockCode), ())
ORDER BY CustomerId DESC, stockCode DESC 
""").show(10)

+----------+---------+-------------+
|customerId|stockCode|sum(Quantity)|
+----------+---------+-------------+
|     18287|    85173|           48|
|     18287|   85040A|           48|
|     18287|   85039B|          120|
|     18287|   85039A|           96|
|     18287|    84920|            4|
|     18287|    84584|            6|
|     18287|   84507C|            6|
|     18287|   72351B|           24|
|     18287|   72351A|           24|
|     18287|   72349B|           60|
+----------+---------+-------------+
only showing top 10 rows



# Joins (8)